In [1]:
import scipy as sp

In [5]:
data = sp.genfromtxt('web_traffic.tsv', delimiter="\t")

In [8]:
print(data.shape)

(743, 2)


In [9]:
print(data[:5])

[[1.000e+00 2.272e+03]
 [2.000e+00       nan]
 [3.000e+00 1.386e+03]
 [4.000e+00 1.365e+03]
 [5.000e+00 1.488e+03]]
